In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import numpy as np
import random
from sklearn.cross_validation import cross_val_score # K折交叉验证模块
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import svm
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.preprocessing import Normalizer
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.multiclass import OutputCodeClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.pipeline import Pipeline as ImPineline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.grid_search import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import lfilter
import pywt

import time
%matplotlib inline

/Users/cql/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/cql/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# spec = total_train_feature.iloc[:,:-1].head()

def specLineNum(spec, windowSize=2, threshold=0.8):
    fs = pd.Series()
    ss = pd.Series()
    for i in range(windowSize, spec.shape[1]):
        s = spec.iloc[:,i-windowSize]
        e = spec.iloc[:,i]
        isFs = (e>threshold) & ((e-s)>threshold)
        isSS = (e<threshold) & ((e-s)<-threshold)
        if(fs.size==0):
            fs = isFs.astype(int)
        else:
            fs = fs + isFs.astype(int)
        if(ss.size==0):
            ss = isSS.astype(int)
        else:
            ss = ss + isSS.astype(int)
    combine = pd.concat([fs, ss], axis=1)
    combine.columns = ['fs','ss']
    return combine

In [3]:
def waveletFilter(x, wavelet="db4", level=1, mode="hard", scaleNum=0.5):
    coeff = pywt.wavedec( x, wavelet, mode="per", axis=1 )
    uthresh = scaleNum * np.sqrt( 2*np.log( x.shape[1] ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode=mode ) for i in coeff[1:] )
    coeff
    # # reconstruct the signal using the thresholded coefficients
    y = pywt.waverec( coeff, wavelet, mode="per" )
    y = pd.DataFrame(y)
    return y

In [4]:
def qsoFeature(data):
    std1 = data.iloc[:,850:880].std(axis=1)
    std2 = data.iloc[:,885:898].std(axis=1)
    std3 = data.iloc[:,900:1000].std(axis=1)
    f1 = (std1<0.3) & (std2>0.35) & (std3<0.35)
    f3 = (std2*std2)/(std1*std3+0.1)
    d1 = std1+0.2
    f2 = (std2>d1)
    qsof = pd.concat([f1, f2, f3], axis=1)
    qsof.columns = ['qso_f1','qso_f2','qso_f3']
    return (qsof)

In [5]:
def intervalEWFeature(d, start=0, end=-1, filterThrehold=1):
    cp = d
#     cp[abs(cp)<filterThrehold] = 0
    rd = cp.iloc[:, start:end]
    specLines = specLineNum(rd[abs(rd)>filterThrehold])
    ew_absorb = rd[(rd<0)&(abs(rd)>filterThrehold)].sum(axis=1)
    ew_fs = rd[(rd>0)&(abs(rd)>filterThrehold)].sum(axis=1)
    return pd.concat([ew_absorb, ew_fs, specLines], axis=1)

def smooth(filterSet):
    n = 5  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    filterSet = lfilter(b, a, filterSet, axis=1)
    return pd.DataFrame(filterSet)

def EWByRangeList(d, rangeList, filterThrehold=1, isSmooth=False, doScale=True):
    if doScale:
        d = pd.DataFrame(scale(d, axis=1))
    if isSmooth:
        d=smooth(d)
    ewfAll = pd.DataFrame()
    for r in rangeList:
        start, end = r
        ewf = intervalEWFeature(d, start, end, filterThrehold)
        newNames = ['ew_' + str(start) + '_' + str(end) + '_' + str(v) for v in ewf.columns.values]
        ewf.columns = newNames
        ewfAll = pd.concat([ewfAll,ewf], axis=1)
    return ewfAll

In [6]:
def submit(fileName, testTotal, preds):
    submission = testTotal[['id']]
    submission['pred'] = preds
    submission.to_csv('./predicts/'+fileName, header=False, index=False)
    return submission
    

In [7]:
def extractFeature(train_feature, partition=2, randomPartion=False, prefix="p", onlyAvg=False, filterLow=False, levelFeature=False, scaleSpec=False, smooth=False):
    if scaleSpec:
        train_feature = pd.DataFrame(scale(train_feature, axis=1))
    if smooth:
        train_feature = train_feature.rolling(100, axis=1).mean()
    if filterLow:
        train_feature[train_feature<1] = 0
    plen = train_feature.shape[1]/partition
    features = pd.DataFrame()
    avgs = pd.DataFrame()
    for i in range(0, partition):
        pstart = random.randint(0, train_feature.shape[1]-plen-1)
        pendExclue = pstart + plen + 1
        avgC = train_feature.iloc[:, pstart:pendExclue].mean(axis=1)
        stdC = train_feature.iloc[:, pstart:pendExclue].std(axis=1)
        maxC = train_feature.iloc[:, pstart:pendExclue].max(axis=1)
        minC = train_feature.iloc[:, pstart:pendExclue].min(axis=1)
        medianC = train_feature.iloc[:, pstart:pendExclue].median(axis=1)
        diffC = train_feature.iloc[:, pstart:pendExclue].diff(axis=1).iloc[:,1:]
        features[prefix+'_avg'+str(i)] = avgC
        avgs[prefix+'_avg'+str(i)] = avgC
        if onlyAvg==False:
            features[prefix+'_std'+str(i)] = stdC
            features[prefix+'_median'+str(i)] = medianC
            features[prefix+'_max'+str(i)] = maxC
            features[prefix+'_min'+str(i)] = minC
            features[prefix+'_avg_m_std'+str(i)] = stdC/avgC
            features[prefix+'_max_m_avg'+str(i)] = maxC/avgC
            features[prefix+'_min_m_avg'+str(i)] = minC/avgC
#             features[prefix+'_media_m_avg'+str(i)] = medianC/avgC
    if levelFeature:
        level_feature = extractFeature(avgs, prefix="l", partition = 4)
        features = pd.concat([features, level_feature], axis = 1)
    return features.fillna(0)

In [8]:
def readTotalFeature(fileName, maxFileIndex, hasLabel=True, partition = 20, ewFeature=True, addQsoFeature=True, doWavlet=False):
    total_train_feature = pd.DataFrame()
    for i in range(0, maxFileIndex+1):
        print('processing:' + str(i))
        train_set = pd.read_csv(fileName + str(i) +'.csv', header=None)
        renameCol = {0:'id'}
        if hasLabel:
            renameCol[train_set.shape[1]-1] = 'label'
        train_set.rename(columns=renameCol, inplace=True)
        if hasLabel:
            processed_train = pd.DataFrame()
            rawFeature = train_set.iloc[:,1:-1]
        else:
            rawFeature = train_set.iloc[:,1:]
        if doWavlet:
            rawFeature = waveletFilter(rawFeature, scaleNum=1)
        processed_train = extractFeature(rawFeature, partition, levelFeature=False, smooth=False)
        if ewFeature:
            rangeList = [(0,300), (0, 500), (500, 1000),(850, 1000), (1000, 1300),(1500,2000), (1500, 2600), (2500, 2600)]
            ew_feature = EWByRangeList(rawFeature, rangeList, filterThrehold=0.5, doScale=True)
            processed_train = pd.concat([processed_train, ew_feature], axis=1)
        if addQsoFeature:
            qso_feature = qsoFeature(rawFeature)
            processed_train = pd.concat([processed_train, qso_feature], axis=1)
        if hasLabel:
            processed_train['label'] = train_set.label
        else:
            processed_train['id'] = train_set.id
        total_train_feature = total_train_feature.append(processed_train, ignore_index=True)
    return total_train_feature.sample(total_train_feature.shape[0], replace=False)

In [9]:
validMode = True
reFeature = True
partition = 40
rng = np.random.RandomState(42)

In [10]:
if reFeature:
    total_train_feature = readTotalFeature('./train_feature_all_', 30, partition=partition)
    total_train_feature.to_csv('./train_feature_check_point.csv', index=False)
else :
    total_train_feature = pd.read_csv('./train_feature_check_point.csv')

processing:0


/Users/cql/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


processing:1
processing:2
processing:3
processing:4
processing:5
processing:6
processing:7
processing:8
processing:9
processing:10
processing:11
processing:12
processing:13
processing:14
processing:15
processing:16
processing:17
processing:18
processing:19
processing:20
processing:21
processing:22
processing:23
processing:24
processing:25
processing:26
processing:27
processing:28
processing:29
processing:30


In [11]:
total_train_feature = total_train_feature.fillna(0)

In [12]:
total_train_feature[total_train_feature.label=='unknown'].head(1000)

,p_avg0,p_std0,p_median0,p_max0,p_min0,p_avg_m_std0,p_max_m_avg0,p_min_m_avg0,p_avg1,p_std1,...,ew_1500_2600_fs,ew_1500_2600_ss,ew_2500_2600_0,ew_2500_2600_1,ew_2500_2600_fs,ew_2500_2600_ss,qso_f1,qso_f2,qso_f3,label
88881,7.322377,11.491131,7.974519,33.697018,-23.246630,1.569317,4.601923,-3.174738,8.243733,2.292599,...,0,6,-93.322591,0.000000,0,0,False,False,0.347547,unknown
69880,295.422757,29.531848,295.495197,352.985047,223.431131,0.099965,1.194847,0.756310,264.648689,37.308614,...,5,22,-138.192817,0.000000,0,5,False,False,0.819199,unknown
195599,19.324678,1.390128,19.392996,23.306534,16.521830,0.071935,1.206050,0.854960,0.160375,7.200709,...,6,7,-1.276416,41.339092,1,2,False,True,12.485280,unknown
9927,4.430249,0.879979,4.367310,6.104386,2.277222,0.198630,1.377888,0.514017,-1.559433,2.233880,...,16,20,-13.509637,28.926765,1,5,False,True,5.518321,unknown
276107,0.360188,1.221538,0.332187,2.833433,-3.307505,3.391393,7.866544,-9.182724,18.478504,4.133943,...,0,0,-51.756161,0.000000,0,0,False,True,1.245206,unknown
47933,25.302543,8.298125,25.209463,58.999094,4.677586,0.327956,2.331746,0.184866,53.536198,16.958524,...,0,0,-63.815025,0.000000,0,0,False,True,2.073894,unknown
19752,59.976250,7.446188,59.800132,77.264077,44.159140,0.124152,1.288245,0.736277,75.089347,5.106326,...,0,0,-81.367980,0.000000,0,0,False,True,1.766905,unknown
224813,-0.457951,6.432202,-0.362030,18.352883,-12.468688,-14.045624,-40.076122,27.227147,-2.641862,3.229105,...,0,0,-0.558804,6.933270,0,0,False,False,0.645146,unknown
124262,10.916965,3.370317,10.923143,22.812146,4.485076,0.308723,2.089605,0.410835,12.235135,4.202329,...,4,6,-10.498420,6.207759,2,1,False,True,19.594353,unknown
230202,50.132039,13.591964,50.465846,78.124827,15.024043,0.271123,1.558381,0.299689,58.059983,10.070315,...,2,0,0.000000,371.221644,2,0,False,True,9.099070,unknown


In [ ]:
X = total_train_feature.iloc[:,:-1]
Y = total_train_feature.iloc[:,-1]

# Split the dataset in two equal parts
x, t_x, y, t_y = train_test_split(
    X, Y, test_size=0.3, random_state=0)
x.shape


(217000, 355)

In [ ]:
if validMode:
    smote = SMOTE()
    parameters = {
        'max_depth':[20, 60, 70, 80],  #70
        'max_features':[0.5,0.6,0.7], #0.6
        'min_samples_split':[2, 4, 8, 20], #2
        'min_samples_leaf':[1,4,8,20], #1
        'min_weight_fraction_leaf':[0,0.05,0.1,0.2], #0
        'max_leaf_nodes':[800, 1000, 1200], #1000
         'bootstrap':[True, False], #False
    #     'oob_score':[True, False], #False
    #     'class_weight':[None,'balanced'], #balanced
        'n_estimators':[10, 20, 100], #10
    #     'criterion':['gini','entropy']
    }
    clf = RandomizedSearchCV(RandomForestClassifier(n_jobs=8, class_weight='balanced'), parameters, cv=3,
                           scoring='f1_macro', n_jobs=8, verbose=2)
    clf.fit(x, y)
    print(clf.best_params_)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] max_leaf_nodes=1000, bootstrap=False, min_samples_leaf=1, n_estimators=10, min_samples_split=8, min_weight_fraction_leaf=0, max_features=0.6, max_depth=70 
[CV] max_leaf_nodes=1000, bootstrap=False, min_samples_leaf=1, n_estimators=10, min_samples_split=8, min_weight_fraction_leaf=0, max_features=0.6, max_depth=70 
[CV] max_leaf_nodes=1000, bootstrap=False, min_samples_leaf=1, n_estimators=10, min_samples_split=8, min_weight_fraction_leaf=0, max_features=0.6, max_depth=70 
[CV] max_leaf_nodes=1000, bootstrap=True, min_samples_leaf=1, n_estimators=20, min_samples_split=8, min_weight_fraction_leaf=0.2, max_features=0.5, max_depth=80 
[CV] max_leaf_nodes=1000, bootstrap=True, min_samples_leaf=1, n_estimators=20, min_samples_split=8, min_weight_fraction_leaf=0.2, max_features=0.5, max_depth=80 
[CV] max_leaf_nodes=1000, bootstrap=True, min_samples_leaf=1, n_estimators=20, min_samples_split=8, min_weight_fraction_leaf=0.2, ma

In [ ]:
smote = SMOTE()
anova_filter = SelectKBest(f_classif, k=(int)(x.shape[1]*0.9))
model = RandomForestClassifier(n_jobs=8, class_weight='balanced', max_depth=20,   verbose=1,random_state=rng)
# model = AdaBoostClassifier(random_state=rng)
# model = GradientBoostingClassifier(random_state=rng,verbose=1)
# model = XGBClassifier()
# model = KNeighborsClassifier(n_neighbors = 3, n_jobs=-1)
scaler = StandardScaler()
clf = ImPineline([
#     ('anova', anova_filter),
    ('scaler',scaler), 
    ('smote',smote),
    ('model', model)
])


In [ ]:
if validMode:
    rs = clf.fit(x, y)
    preds_test = clf.predict(t_x)
#     preds_test = carbateQso(t_x, preds_test)
#     clf.score(t_x, t_y)
    print("%0.3f" % f1_score(t_y, preds_test, average='macro'))  
    print(classification_report(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))
    print(confusion_matrix(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))

In [ ]:
def carbateQso(t_x, preds_test):
    fPreds = pd.concat([t_x.reset_index(), pd.DataFrame(preds_test)], axis=1)
    fPreds[(fPreds.iloc[:,-1]=='unknown')&(fPreds.qso_f3>2)].iloc[:,-1] = 'qso'
    return fPreds.iloc[:,-1].values

In [ ]:
if validMode:
    clf = RandomForestClassifier(n_jobs=-1, class_weight='balanced',  verbose=1 )
    rs = clf.fit(x, y)
    preds_test = clf.predict(t_x)
    # clf.score(t_x, t_y)
    print('f1: %.2f' % f1_score(t_y, preds_test, average='macro')) 
    print(classification_report(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))
    print(confusion_matrix(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))

In [ ]:
if validMode:
    rs = clf.fit(x, y)
    scores = cross_val_score(clf, x, y, cv=3, n_jobs=-1, verbose=1, scoring='f1_macro')
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#     preds_test = clf.predict(t_x)
    # clf.score(t_x, t_y)
#     print("%0.3f" % f1_score(t_y, preds_test, average='macro'))  

In [ ]:
if validMode:
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])
    plt.show()


In [ ]:
if reFeature:
    testTotal = readTotalFeature('./test_feature_all_', 9, hasLabel=False, partition=partition)
    testTotal.to_csv('./test_feature_checkpoint.csv', index=False)
    testTotal.head()
else :
    testTotal = pd.read_csv('./test_feature_checkpoint.csv')

In [ ]:
testTotal.shape

In [ ]:
X.shape

In [ ]:
clf.fit(X, Y)



In [ ]:
test_x = testTotal.iloc[:,:-1]
pred = clf.predict(test_x)
timestr = time.strftime("%m%d_%H%M")
rs = submit(timestr+'_rf_ew_40.csv',testTotal, pred)


In [ ]:
rs.pred.value_counts()

In [ ]:
total_train_feature.head()

In [ ]:
total_train_feature[:,:-1]